In [ ]:
#############################################################################################################################################
# Player (1) vs AI (0) - nxn Determinant Game with Randomly chosing when no pre-"good shapes" 
#############################################################################################################################################
import numpy as np

n = int(input('Enter grid size (n): '))
matrix = np.full((n, n), None)  # None represents empty cells

def get_move(used):
    while True:
        try:
            move = input(f"Enter position (row,col 1-{n}): ")
            row, col = map(int, move.split(','))
            row -= 1
            col -= 1
            
            if not (0 <= row < n and 0 <= col < n):
                print(f"Input out of range. Use values 1-{n}")
                continue
                
            if (row, col) in used:
                print("Position already occupied")
                continue
                
            return (row, col)
            
        except (ValueError, IndexError):
            print("Invalid format. Use: row,col (e.g. 2,3)")

def print_grid(grid):
    print("\n   " + " ".join(str(i+1).rjust(2) for i in range(n)))  # Column headlines
    for i in range(n):
        row = [' .' if cell is None else f' {int(cell)}' for cell in grid[i]]
        print(f"{i+1:2} " + " ".join(row))  # Row number + content
def game(start_player):
    used = set()
    current = start_player
    while len(used) < n*n:
        print_grid(matrix)
        print(f"\n[{current}'s turn]")
        
        if current == 'Human':
            row, col = get_move(used)
            matrix[row][col] = 1
        else:
            # AI strategy: try to create linear dependencies
            # 1. Check for existing 0-filled rows/columns
            best_move = None
            for i in range(n):
                # Check rows
                row = matrix[i, :]
                if np.sum(row == 0) > 0 and None in row:
                    col_idx = np.where(row == None)[0][0]
                    best_move = (i, col_idx)
                    break
                # Check columns
                col = matrix[:, i]
                if np.sum(col == 0) > 0 and None in col:
                    row_idx = np.where(col == None)[0][0]
                    best_move = (row_idx, i)
                    break          
            # 2. If no good move found, choose random ##COULD ADD ON ADVANCE ALGORITHM
            if best_move is None:
                empty = np.argwhere(matrix == None)
                best_move = tuple(empty[np.random.choice(len(empty))])           
            row, col = best_move
            matrix[row][col] = 0
            print(f"AI places 0 at ({row+1}, {col+1})")
        
        used.add((row, col))
        current = 'AI' if current == 'Human' else 'Human'
## Determiant calculation
    print_grid(matrix)
    filled = np.where(matrix == None, 1, matrix).astype(float)
    det = np.linalg.det(filled)
    
    print(f"\nFinal determinant = {det:.2f}")
    if abs(det) > 1e-6:
        print("Human wins!")
    else:
        print("AI wins!")

## Start game
starter = input("Who starts first? (Human/AI): ").strip().capitalize()
game(starter if starter in ('Human', 'AI') else 'Human')

In [1]:
#############################################################################################################################################
# Player (1) vs AI (0) - nxn Determinant Game with Minimax algorithm
#############################################################################################################################################
import numpy as np

n = int(input('Enter grid size (n): '))
matrix = np.full((n, n), None)

def get_move(used):
    while True:
        try:
            move = input(f"Enter position (row,col 1-{n}): ")
            row, col = map(int, move.split(','))
            row -= 1
            col -= 1
            
            if not (0 <= row < n and 0 <= col < n):
                print(f"Input out of range. Use values 1-{n}")
                continue
                
            if (row, col) in used:
                print("Position already occupied")
                continue
                
            return (row, col)
            
        except (ValueError, IndexError):
            print("Invalid format. Use: row,col (e.g. 2,3)")

def print_grid(grid):
    print("\n   " + " ".join(str(i+1).rjust(2) for i in range(n)))
    for i in range(n):
        row = [' .' if cell is None else f' {int(cell)}' for cell in grid[i]]
        print(f"{i+1:2} " + " ".join(row))

def evaluate(board):
    filled = np.where(board == None, 1, board).astype(float)
    det = np.linalg.det(filled)
    return 1 if abs(det) > 1e-6 else -1

def minimax(board, turn):
    moves = [(i, j) for i in range(n) for j in range(n) if board[i][j] is None]
    if not moves: 
        return evaluate(board), None

    if turn == "Human":
        bestScore = -float('inf')
        bestMove = None
        for move in moves:
            new_board = board.copy()
            i, j = move
            new_board[i][j] = 1
            score, _ = minimax(new_board, "AI")
            if score > bestScore:
                bestScore = score
                bestMove = move
        return bestScore, bestMove
    else:  # AI's turn (minimizing player)
        bestScore = float('inf')
        bestMove = None
        for move in moves:
            new_board = board.copy()
            i, j = move
            new_board[i][j] = 0
            score, _ = minimax(new_board, "Human")
            if score < bestScore:
                bestScore = score
                bestMove = move
        return bestScore, bestMove

def game(start_player):
    used = set()
    current = start_player
    while len(used) < n * n:
        print_grid(matrix)
        print(f"\n[{current}'s turn]")
        
        if current == "Human":
            # Get the human move.
            row, col = get_move(used)
            matrix[row][col] = 1
        else:
            # AI uses minimax to determine the best move.
            print("AI is thinking...")
            _, move = minimax(matrix, "AI")
            if move is None:
                # Fallback (should not occur if board is not full)
                empty = np.argwhere(matrix == None)
                move = tuple(empty[np.random.choice(len(empty))])
            row, col = move
            matrix[row][col] = 0
            print(f"AI places 0 at ({row+1}, {col+1})")
        
        used.add((row, col))
        current = "AI" if current == "Human" else "Human"
    print_grid(matrix)
    filled = np.where(matrix == None, 1, matrix).astype(float)
    det = np.linalg.det(filled)
    print(f"\nFinal determinant = {det:.2f}")
    if abs(det) > 1e-6:
        print("Human wins!")
    else:
        print("AI wins!")

# Start game
starter = input("Who starts first? (Human/AI): ").strip().capitalize()
if starter not in ("Human", "Ai"):
    starter = "Human"
game(starter)

Enter grid size (n): 3
Who starts first? (Human/AI): human

    1  2  3
 1  .  .  .
 2  .  .  .
 3  .  .  .

[Human's turn]
Enter position (row,col 1-3): 2,2

    1  2  3
 1  .  .  .
 2  .  1  .
 3  .  .  .

[AI's turn]
AI is thinking...
AI places 0 at (1, 1)

    1  2  3
 1  0  .  .
 2  .  1  .
 3  .  .  .

[Human's turn]
Enter position (row,col 1-3): 3,1

    1  2  3
 1  0  .  .
 2  .  1  .
 3  1  .  .

[AI's turn]
AI is thinking...
AI places 0 at (1, 3)

    1  2  3
 1  0  .  0
 2  .  1  .
 3  1  .  .

[Human's turn]
Enter position (row,col 1-3): 1,2

    1  2  3
 1  0  1  0
 2  .  1  .
 3  1  .  .

[AI's turn]
AI is thinking...
AI places 0 at (2, 3)

    1  2  3
 1  0  1  0
 2  .  1  0
 3  1  .  .

[Human's turn]
Enter position (row,col 1-3): 2,1

    1  2  3
 1  0  1  0
 2  1  1  0
 3  1  .  .

[AI's turn]
AI is thinking...
AI places 0 at (3, 3)

    1  2  3
 1  0  1  0
 2  1  1  0
 3  1  .  0

[Human's turn]
Enter position (row,col 1-3): 3,2

    1  2  3
 1  0  1  0
 2  1  1  0
 

In [ ]:
#############################################################################################################################################
# Player (1) vs AI (0) - nxn Determinant Game with Minimax algorithm and Alpha-Beta pruning
#############################################################################################################################################
import numpy as np

n = int(input('Enter grid size (n): '))
matrix = np.full((n, n), None)

def get_move(used):
    while True:
        try:
            move = input(f"Enter position (row,col 1-{n}): ")
            row, col = map(int, move.split(','))
            row -= 1
            col -= 1
            
            if not (0 <= row < n and 0 <= col < n):
                print(f"Input out of range. Use values 1-{n}")
                continue
                
            if (row, col) in used:
                print("Position already occupied")
                continue
                
            return (row, col)
            
        except (ValueError, IndexError):
            print("Invalid format. Use: row,col (e.g. 2,3)")

def print_grid(grid):
    print("\n   " + " ".join(str(i+1).rjust(2) for i in range(n)))
    for i in range(n):
        row = [' .' if cell is None else f' {int(cell)}' for cell in grid[i]]
        print(f"{i+1:2} " + " ".join(row))

def evaluate(board):
    filled = np.where(board == None, 1, board).astype(float)
    det = np.linalg.det(filled)
    return 1 if abs(det) > 1e-6 else -1

def minimax(board, turn, alpha=-float('inf'), beta=float('inf')):
    moves = [(i, j) for i in range(n) for j in range(n) if board[i][j] is None]
    if not moves:  
        return evaluate(board), None

    if turn == "Human":
        bestScore = -float('inf')
        bestMove = None
        for move in moves:
            new_board = board.copy()
            i, j = move
            new_board[i][j] = 1
            score, _ = minimax(new_board, "AI", alpha, beta)
            if score > bestScore:
                bestScore = score
                bestMove = move
            alpha = max(alpha, bestScore)
            if beta <= alpha:  # Pruning Condition
                break
        return bestScore, bestMove
    else:
        bestScore = float('inf')
        bestMove = None
        for move in moves:
            new_board = board.copy()
            i, j = move
            new_board[i][j] = 0
            score, _ = minimax(new_board, "Human", alpha, beta)
            if score < bestScore:
                bestScore = score
                bestMove = move
            beta = min(beta, bestScore)
            if beta <= alpha:  # Pruning Condition
                break
        return bestScore, bestMove

def game(start_player):
    used = set()
    current = start_player
    while len(used) < n * n:
        print_grid(matrix)
        print(f"\n[{current}'s turn]")
        
        if current == "Human":
            row, col = get_move(used)
            matrix[row][col] = 1
        else:
            print("AI is thinking...")
            _, move = minimax(matrix, "AI")
            if move is None:
                empty = np.argwhere(matrix == None)
                move = tuple(empty[np.random.choice(len(empty))])
            row, col = move
            matrix[row][col] = 0
            print(f"AI places 0 at ({row+1}, {col+1})")
        
        used.add((row, col))
        current = "AI" if current == "Human" else "Human"
    print_grid(matrix)
    filled = np.where(matrix == None, 1, matrix).astype(float)
    det = np.linalg.det(filled)
    print(f"\nFinal determinant = {det:.2f}")
    if abs(det) > 1e-6:
        print("Human wins!")
    else:
        print("AI wins!")
starter = input("Who starts first? (Human/AI): ").strip().capitalize()
if starter not in ("Human", "Ai"):
    starter = "Human"
game(starter)

Enter grid size (n): 4
Who starts first? (Human/AI): human

    1  2  3  4
 1  .  .  .  .
 2  .  .  .  .
 3  .  .  .  .
 4  .  .  .  .

[Human's turn]
Enter position (row,col 1-4): 1,1

    1  2  3  4
 1  1  .  .  .
 2  .  .  .  .
 3  .  .  .  .
 4  .  .  .  .

[AI's turn]
AI is thinking...
AI places 0 at (1, 2)

    1  2  3  4
 1  1  0  .  .
 2  .  .  .  .
 3  .  .  .  .
 4  .  .  .  .

[Human's turn]
